# ConversationBufferMemory

- Author: [Sungchul Kim](https://github.com/rlatjcj)
- Peer Review : [Shinar12](https://github.com/Shinar12), [BAEM1N](https://github.com/BAEM1N), [YellowGangneng](https://github.com/YellowGangneng)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/01-ConversationBufferMemory.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/01-ConversationBufferMemory.ipynb)

## Overview

This tutorial introduces `ConversationBufferMemory`, a memory class that stores conversation history in a buffer.

Typically, no additional processing is required. Sometimes, however, it may be necessary when the conversation history exceeds the model's context window.

In this tutorial, we will learn how to use `ConversationBufferMemory` to store and retrieve conversation history.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Extracting messages as strings](#extracting-messages-as-strings)
- [Extracting messages as HumanMessage and AIMessage objects](#extracting-messages-as-humanmessage-and-aimessage-objects)
- [Applying to a Chain](#applying-to-a-chain)

### References
- [LangChain: ConversationBufferMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "ConversationBufferMemory",
    }
)

Environment variables have been set successfully.


You can alternatively set `OPENAI_API_KEY` in `.env` file and load it.

**[Note]**  
- This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [6]:
from dotenv import load_dotenv

load_dotenv()

False

## Extracting messages as strings

After storing conversation messages, this memory allows you to extract messages into a variable.


In [7]:
from langchain.memory import ConversationBufferMemory

In [8]:
#ConversationBufferMemory는 전체 대화 기록을 저장하는 메모리 방식
memory = ConversationBufferMemory()
memory

/var/folders/nj/q5g56j4x2qjdw_n8_f5l1w0m0000gn/T/ipykernel_70475/2223904900.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

You can use the `save_context(inputs, outputs)` method to save conversation records.

- This method accepts two arguments, `inputs` and `outputs`.
- `inputs` stores the user's question, and `outputs` stores the AI's answer.
- The conversation record is stored internally under the `history` key.
- You can then use the `load_memory_variables` method to retrieve and inspect the saved conversation history.


In [9]:
# inputs: dictionary(key: "human" or "ai", value: question)
# outputs: dictionary(key: "ai" or "human", value: answer)
#사용자의 질문과 AI의 응답을 memory.save_context()를 사용해 저장

memory.save_context(
    inputs={
        "human": "Hello, I want to open a bank account remotely. How do I start?",
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification."
    },
)

In [10]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, I want to open a bank account remotely. How do I start?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.", additional_kwargs={}, response_metadata={})]))

The `load_memory_variables({})` method of the memory object returns the complete message history as a string.

In [11]:
# Check the message history stored in the 'history' key.
#memory.load_memory_variables({})["history"]를 이용해 대화 내역 출력
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.


In [12]:
memory.save_context(
    inputs={
        "human": "Yes, I've prepared my ID for identity verification. What should I do next?"
    },
    outputs={
        "ai": "Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next."
    },
)

In [13]:
# Check the message history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.


In [14]:
# Save 2 conversations.
memory.save_context(
    inputs={
        "human": "I uploaded the photo. How do I proceed with identity verification?"
    },
    outputs={
        "ai": "We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text."
    },
)
memory.save_context(
    inputs={
        "human": "I entered the verification number. How do I open an account now?"
    },
    outputs={
        "ai": "Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc."
    },
)

In [15]:
# Check the conversation history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.
Human: I uploaded the photo. How do I proceed with identity verification?
AI: We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.
Human: I entered the verification number. How do I open an account now?
AI: Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc.


In [16]:
# Save 2 more conversations.
memory.save_context(
    inputs={
        "human": "I've entered all the information. What's the next step?",
    },
    outputs={
        "ai": "I've confirmed the information you've entered. The account opening process is almost complete. Please agree to the terms of use and confirm the account opening."
    },
)
memory.save_context(
    inputs={
        "human": "I've completed all the steps. Has the account been opened?",
    },
    outputs={
        "ai": "Yes, the account has been opened. Your account number and related information have been sent to the email you registered. If you need additional help, please contact us at any time. Thank you!"
    },
)

In [17]:
# Check the conversation history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.
Human: I uploaded the photo. How do I proceed with identity verification?
AI: We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.
Human: I entered the verification number. How do I open an account now?
AI: Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc.
Human: I've entered all the information. What's the next step?
AI: I've confirmed the information you've entered. The accoun

## Extracting messages as `HumanMessage` and `AIMessage` objects

Setting `return_messages=True` returns `HumanMessage` and `AIMessage` objects.

In [18]:
#리스트 방식 출력
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "Hello, I want to open a bank account remotely. How do I start?",
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.",
    },
)

memory.save_context(
    inputs={
        "human": "Yes, I've prepared my ID for identity verification. What should I do next?"
    },
    outputs={
        "ai": "Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next."
    },
)

memory.save_context(
    inputs={
        "human": "I uploaded the photo. How do I proceed with identity verification?"
    },
    outputs={
        "ai": "We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text."
    },
)

In [19]:
# Check the conversation history stored in the 'history' key.
memory.load_memory_variables({})["history"]

[HumanMessage(content='Hello, I want to open a bank account remotely. How do I start?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Yes, I've prepared my ID for identity verification. What should I do next?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I uploaded the photo. How do I proceed with identity verification?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.', additional_kwargs={}, response_metadata=

## Applying to a Chain

Let's apply `ConversationBufferMemory` to a `ConversationChain`.

In [20]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

# Create an LLM model.
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create a ConversationChain.
conversation = ConversationChain(
    # Use ConversationBufferMemory.
    llm=llm,
    memory=ConversationBufferMemory(),
)

/var/folders/nj/q5g56j4x2qjdw_n8_f5l1w0m0000gn/T/ipykernel_70475/3549519840.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


Proceed with the conversation using the `ConversationChain`.

In [21]:
# Start the conversation.
response = conversation.predict(
    input="Hello, I want to open a bank account remotely. How do I start?"
)
print(response)

Hello! Opening a bank account remotely is quite convenient these days, thanks to digital banking services. Here’s a step-by-step guide to help you get started:

1. **Research Banks**: First, decide which bank or credit union you want to open an account with. Consider factors like fees, interest rates, customer service, and the availability of online banking features. Many banks offer online-only accounts with lower fees.

2. **Visit the Bank’s Website**: Once you’ve chosen a bank, visit their official website. Look for an option like “Open an Account” or “Apply Now.” This is usually prominently displayed on the homepage.

3. **Choose the Type of Account**: Decide whether you want a checking account, savings account, or both. Some banks offer special accounts for students, seniors, or businesses, so choose the one that best fits your needs.

4. **Prepare Your Documents**: You’ll typically need to provide some personal information and documents. This usually includes:
   - A government-i

Verify if the system remembers the previous conversation.

In [22]:
# Send a request to summarize the previous conversation in bullet points.
response = conversation.predict(input="Summarize the previous answer in bullet points.")
print(response)

- **Research Banks**: Choose a bank based on fees, interest rates, and online features.
- **Visit the Bank’s Website**: Look for “Open an Account” or “Apply Now.”
- **Choose the Type of Account**: Decide between checking, savings, or specialized accounts.
- **Prepare Your Documents**: Have your ID, Social Security number, proof of address, and employment info ready.
- **Fill Out the Application**: Enter personal and contact details accurately.
- **Verify Your Identity**: Upload ID, answer security questions, or have a video call.
- **Fund Your Account**: Make an initial deposit via transfer, credit card, or mobile payment.
- **Set Up Online Banking**: Manage your account online.
- **Receive Your Debit Card**: Expect it in the mail if you opened a checking account.
